In [1]:
# 1. importing useful libraries

import requests 
import time     
import re       
from bs4 import BeautifulSoup 

In [2]:
# access the webpage as Chrome
data = []
my_headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}

In [3]:
data = []
movie = 'donnie_darko'
pageNum = 4

for k in range(1,pageNum+1):
    
   
    page = 'https://www.rottentomatoes.com/m/'+str(movie)+'/reviews?type=&sort=1&page='+str(k) 
   
    src  = False

    for i in range(1,6):
        try:
            response = requests.get(page, headers = my_headers)
            src = response.content
            break 
        except:
            print ('failed attempt #',i)
            time.sleep(2)

    if not src:
        print('Could not get page: ', page)
        continue 
    else:
        print('Successfully got page: ', page)

    text = response.text        
        
    soup = BeautifulSoup(src.decode('ascii', 'ignore'), 'lxml')
    review_rows = soup.find_all('div', {'class':re.compile('row review_table_row')})

    for review_row in review_rows:


        # 1) find a, which has two keys (href and class)
        critic_name = review_row.find('a',{'class':'unstyled bold articleLink'})
        
        if critic_name:
            critic_name = critic_name.text
        else: critic_name = 'NA' # In case HTML code with given class is missing

            
        # 2) find div with the class that starts with "review_icon icon small..." and grap the third element
        rating = review_row.find('div',{'class':re.findall('review_icon icon small (.*)">', text)})

        if rating:
            rating = rating.attrs['class'][3]
        else: rating  = 'NA' # In case HTML code with given class is missing
        
        
        # 3) find em and em's text
        source = review_row.find('em')
        
        if source:
            source = source.text
        else: source = 'NA' # In case HTML code with 'em' is missing
            
            
        # 4) find div with the class "the_review", then its text and strip it
        review_text = review_row.find('div',{'class':'the_review'})
        
        if review_text.text.strip() == '': #because if text is not there, the HTML code with that class is just empty
            review_text = 'NA'
        else:
            review_text = review_text.text.strip()
        
        
        # 4) find div with the class "review-date subtle small", then its text and strip it
        date = review_row.find('div',{'class':'review-date subtle small'})
        
        if date.text.strip() == '': #because if date is not there, the HTML code with that class is just empty
            date = 'NA'
        else: date = date.text.strip()

            
        # add all the info to the data link
        data.append([critic_name, rating, source, review_text, date])

Successfully got page:  https://www.rottentomatoes.com/m/donnie_darko/reviews?type=&sort=1&page=1
Successfully got page:  https://www.rottentomatoes.com/m/donnie_darko/reviews?type=&sort=1&page=2
Successfully got page:  https://www.rottentomatoes.com/m/donnie_darko/reviews?type=&sort=1&page=3
Successfully got page:  https://www.rottentomatoes.com/m/donnie_darko/reviews?type=&sort=1&page=4


In [4]:
for review_row in data:
    print(review_row)

['Mike Massie', 'rotten', 'Gone With The Twins', 'Every ounce of creativity is met with an equal serving of disappointing nebulousness.', 'September 29, 2020']
['Erik Anderson', 'fresh', 'AwardsWatch', "It's a dense, complicated and amazing amalgam of genres and influences that really defies any constraints.", 'April 7, 2020']
['Rachel Wagner', 'fresh', 'rachelsreviews.net', "I certainly didn't have an emotional bond with the film. Still, I admire the risk taking and what it has to say about mental illness.", 'March 7, 2019']
['PJ Nabarro', 'fresh', 'Patrick Nabarro', "It's ultimately the piece's surprisingly sincere, compassionate edge which transmits the most.", 'November 12, 2018']
['Sean Axmaker', 'fresh', 'Stream on Demand', '[Richard] Kelly winds so many ideas, so many questions, and so much marvelously strange experience through this ambitious film that you can forgive the few loose ends and overdone caricatures amidst the richness.', 'December 16, 2017']
['Ray Pride', 'fresh', 

In [5]:
with open('olha_sverdel_donnie_darko.txt', mode='w', encoding='utf-8') as f:
    for review_row in data:
        f.write(review_row[0] + '\t' + review_row[1] + '\t' + review_row[2] + '\t' + review_row[3] + '\t' + review_row[4] + '\n')